In [1]:
import pandas as pd
import time
import numpy as np

In [2]:
df_desc = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_1_descriptor_combined.tsv")
indep_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/9_2_indep_learning_association.tsv")
indep_drug = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/8_2_indep_standard_drug_binary_matrix.tsv", index_col=0)

In [3]:
# 2017.3.8. 작성. 기존의 gold descriptor를 independet의 feature에 맞춰 수정한다.
indep_drug_feature = indep_drug.index.values
disease_feature = df_desc.columns.values[2279:]

In [8]:
df_desc_drug = df_desc.filter(items = indep_drug_feature)
df_desc_disease = df_desc.filter(items= disease_feature)

In [17]:
frames = [df_desc.loc[:, :'association'], df_desc_drug, df_desc_disease]
df_new_gold_desc = pd.concat(frames, axis = 1)

In [20]:
df_new_gold_desc.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/2_1_2_new_descriptor.tsv", sep='\t', index=False)
df_new_gold_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_1_2_new_descriptor.tsv", sep='\t', index=False)

In [24]:
# 2017.3.8. 작성. 기존의 independent descriptor를 independent feature에 맞춰 수정한다.
df_indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_2_indep_asso_descriptor.tsv")

In [ ]:
df_new_indep_desc = df_indep.filter(items = df_new_gold_desc.columns.values)

In [ ]:
df_new_indep_desc.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/2_2_2_new_indep_asso_descriptor.tsv", sep='\t', index=False)
df_new_indep_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_2_2_new_indep_asso_descriptor.tsv", sep='\t', index=False)

In [ ]:
#snake make 사용을 위한 분할저장

In [25]:
pos = df_new_gold_desc[df_new_gold_desc.association == 1]
neg = df_new_gold_desc[df_new_gold_desc.association == 0]

In [ ]:
pos_index = pos.index
neg_index = neg.index

In [ ]:
from itertools import product
pos_neg=pd.DataFrame(list(product(pos_index, neg_index)), columns = ['positive_index', 'negative_index'])

In [ ]:
print len(pos_neg)
pos_neg.head()

In [ ]:
a = np.array_split(pos_neg, 200)

In [ ]:
for df, i in zip(a, range(200)):
    df.to_csv("/home/share/aschoi/snakemake/jaccard_score/input/2_1_2_new_descriptor_index"+str(i+1)+".tsv", sep='\t', index=False)

In [ ]:
df_new_gold_desc.head(2)

In [ ]:
# 아래는 similarity score구하는거.

In [ ]:
from sklearn.metrics import jaccard_similarity_score

In [22]:
index = pd.read_table("/home/share/aschoi/snakemake/jaccard_score/input/2_1_2_new_descriptor_index1.tsv")

In [ ]:
df_jac = pd.DataFrame()
for index, row in df_index.iterrows():
    pos_index = row[0]
    neg_index = row[1]
    pos_desc = df_new_gold_desc.iloc[pos_index]
    neg_desc = df_new_gold_desc.iloc[neg_index]
    
    jac_sim = jaccard_similarity_score(pos_desc[3:].tolist(), neg_desc[3:].tolist())
    
    temp_df = pd.DataFrame([[pos_desc[0], pos_desc[1], neg_desc[0], neg_desc[1], jac_sim]],
                           columns = ['Pos_Drug', 'Pos_Disease', 'Neg_DRug', 'Neg_disease', 'Sim_score'])
    df_jac = df_jac.append(temp_df)    

In [ ]:
df_jac.to_csv(sys.argv[3], index=False)